# Caracterización de los Usuario en Distribución

## Usuarios de la red de distribución con *'df_clientes'*

El *'df_clientes'* está hecho a partir de la información proporcionada por El Coordinador a través de su base de datos pública. Los datos del consumo facturado corresponden al promedio mensual entre Junio 2024 y Junio 2025 según lo exigido en el Artículo 1-17 de la NTCSD 2024.

La razón de uso de este periodo (Junio 2024 - Junio 2025) se debe a que es la única base de datos con información sobre la comuna del punto de consumo (aunque no esté completa, faltando aproximadamente 1.000.000 de clientes sin información sobre la comuna).

Fuente de los datos: [Página web del Coordinador](https://www.coordinador.cl/mercados/documentos/transferencias-economicas-de-empresas-distribuidoras/catastro-clientes-usuarios-en-distribucion/catastro-2025/)

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint

In [2]:
# Usuarios de la red de Distribución
df_clientes = pd.read_parquet("./Datos_Dx_procesados/ClientesDx.parquet")     # Original

# Se elimina el resto de los registros que contienen nulos en la Comuna
df_clientes_norm = df_clientes.dropna(subset=["Comuna"]).copy()
df_clientes_sin_comuna = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_norm["id_usuario"])]

df_clientes_norm

,Distribuidor,EmpalmeCodigo,ClienteVigencia,ClienteCalidadJuridica,Comuna,ConsumoTipoUsuario,ConsumoTipoConsumo,ConsumoTipoSuministro,ConsumoTipoTarifa,ConsumoTension,ConsumoPotencia,ConsumoFacturado,ConexionAlimentador,ConexionSubestacionPrimariaDistribucion,id_usuario
0,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,44.355,ALIMENTADOR 2 6,PUERTO NATALES,0
1,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,10.645,ALIMENTADOR 2 6,PUERTO NATALES,1
2,edelmag,101694022,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,1.936,ALIMENTADOR 2 4,PUERTO NATALES,2
3,edelmag,101694024,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,4.167,ALIMENTADOR 2 4,PUERTO NATALES,3
4,edelmag,101836414,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,92.742,ALIMENTADOR 2 2,PUERTO NATALES,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884012,electrica til til,56229,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,16.0,709.000,RUNGUE,RUNGUE,7951469
7884013,electrica til til,56232,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,3.0,275.000,TILTIL,POLPAICO (ENEL),7951470
7884014,electrica til til,56233,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,32.0,0.000,TILTIL,POLPAICO (ENEL),7951471
7884015,electrica til til,56244,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,13kV,15.0,17.000,CALEU,CALEU,7951472


In [4]:
# Nulos en columnas de interés
print(" -> Nulos originales en la columna \"Comuna\" en df_clientes = {:,}".format(df_clientes["Comuna"].isna().sum()))

 -> Nulos originales en la columna "Comuna" en df_clientes = 1,125,223


In [5]:
# Distribudoras con nulos en la columna Comuna
# Nulos en total en la columna Comuna = 1,125,223
df_con_nulos = df_clientes_sin_comuna[df_clientes_sin_comuna["Comuna"].isna()].copy()
df_con_nulos["Distribuidor"].value_counts()

Distribuidor
saesa                553753
frontel              442093
edelaysen             55932
enel distribucion     39633
luz osorno            32805
coopersol              1007
Name: count, dtype: int64

### Sobre la baja calidad de los datos del tipo de tarifa en df_clientes asociados a Enel

Se demuestra con dos ejemplos (**hay hartos más**) que la columna con información sobre el tipo de tarifa de los clientes en comunas donde opera "Enel" no tiene sentido. Los datos enviados por esta empresa al CEN quizás están corrompidos o sufrieron algún tipo de modificación, por lo tanto, al momento de hacer la limpieza, no se considerará para estos casos el fitro en el tipo de tarifa de los clientes.

In [6]:
# Ejemplo 1 de los registros mal clsificados en la comuna de San Miguel
df_san_miguel = df_clientes_norm[df_clientes_norm["Comuna"] == "san miguel"]
tipo_de_tarifa = df_san_miguel["ConsumoTipoTarifa"].value_counts(dropna=False)

print(" -> Cantidad de resgistros en la comuna de San Miguel = {:,}".format(len(df_san_miguel)))
print(" -> Empresa/s distribuidora/s que lo abastece/n: ", df_san_miguel["Distribuidor"].unique())
print(" -> Proporción de cada tipo de tarifa en San Miguel:")
pprint(tipo_de_tarifa)

 -> Cantidad de resgistros en la comuna de San Miguel = 67,750
 -> Empresa/s distribuidora/s que lo abastece/n:  ['enel distribucion']
 -> Proporción de cada tipo de tarifa en San Miguel:
ConsumoTipoTarifa
AT2PP          58386
Peaje_AT3PP     8138
<NA>             609
BT2PP            487
Peaje_AT2PP       62
BT1b              27
BT4.2             26
BT2PPP            14
BT3PP              1
Name: count, dtype: Int64


In [7]:
# Ejemplo 2 de los registros mal clsificados en la comuna de Cerrillos
df_cerrillos = df_clientes_norm[df_clientes_norm["Comuna"] == "cerrillos"]
tipo_de_tarifa = df_cerrillos["ConsumoTipoTarifa"].value_counts(dropna=False)

print(" -> Cantidad de resgistros en la comuna de Cerrillos = {:,}".format(len(df_cerrillos)))
print(" -> Empresa/s distribuidora/s que lo abastece/n: ", df_san_miguel["Distribuidor"].unique())
print(" -> Proporción de cada tipo de tarifa en Cerrillos:")
pprint(tipo_de_tarifa)

 -> Cantidad de resgistros en la comuna de Cerrillos = 29,325
 -> Empresa/s distribuidora/s que lo abastece/n:  ['enel distribucion']
 -> Proporción de cada tipo de tarifa en Cerrillos:
ConsumoTipoTarifa
AT2PP          28590
<NA>             222
Peaje_AT3PP      160
BT2PP            152
Peaje_AT2PP      136
BT4.2             50
BT1b              12
BT2PPP             3
Name: count, dtype: Int64


## Combinación: "*df_clientes*" x "*df_par_comuna_empresadx*"

El *'df_par_comuna_empresadx'* está hecho a partir de la información proporcionada por Comisión Nacional de Energía a través de la NTCSD 2024. Realiza una clasificación sobre la densidad de clientes en el par Comuna-Empresa.

Las clasifica en 5 grupos posibles:

 - "EXTREMADAMENTE BAJA" 
 - "MUY BAJA"
 - "BAJA"
 - "MEDIA"
 - "ALTA"

Fuente de los datos: [Norma Técnica de Calidad de Servicio para Sistemas de Distribución](https://www.cne.cl/wp-content/uploads/2024/05/NTCSDx2024-1.pdf) (Anexo: Clasificación de redes)

In [8]:
# Densidad de clientes en el Par Comuna-Empresa de Distribución
df_par_comuna_empresadx = pd.read_excel("./Datos_Dx_procesados/Par-Comuna-EmpresaDx (procesados).xlsx", index_col=0, engine='openpyxl')
df_par_comuna_empresadx

,Comuna,Distribuidor,Densidad,Mapeo_Densidad
0,aisen,edelaysen,MUY BAJA,2
1,algarrobo,edecsa,EXTREMADAMENTE BAJA,1
2,algarrobo,litoral,BAJA,3
3,alhue,cge,EXTREMADAMENTE BAJA,1
4,alto biobio,frontel,EXTREMADAMENTE BAJA,1
...,...,...,...,...
479,bulnes,cge,EXTREMADAMENTE BAJA,1
480,vilcun,cge,EXTREMADAMENTE BAJA,1
481,putre,coopersol,EXTREMADAMENTE BAJA,1
482,arica,desa,EXTREMADAMENTE BAJA,1


### Diferencias entre ambos dataframes

In [9]:
# Elementos únicos en las columnas ascociadas a Distribuidoras y Comunas 
# df_par_comuna_empresadx
distribuidor_par_unique = set(df_par_comuna_empresadx["Distribuidor"].unique())
comuna_par_unique = set(df_par_comuna_empresadx["Comuna"].unique())

# df_clientes_norm
distribuidor_clientes_unique = set(df_clientes_norm["Distribuidor"].unique())
comuna_clientes_unique = set(df_clientes_norm["Comuna"].unique())

In [10]:
# Comunas y Distribuidoras en los que difieren en ambos dataframes
comunas_distintas = comuna_par_unique.symmetric_difference(comuna_clientes_unique)
empresas_distintas = distribuidor_par_unique.symmetric_difference(distribuidor_clientes_unique)

print(" Cantidad de Comunas evaluadas en df_clientes: ", len(comuna_clientes_unique), 
      "\n Cantidad de Comunas en df_par (información CNE): ", len(comuna_par_unique),
      "\n -> Comunas distintas: ", len(comunas_distintas))

print("\n Cantidad de Distribuidoras evaluadas en df_clientes: ", len(distribuidor_clientes_unique), 
      "\n Cantidad de Distribuidoras en df_par (información CNE): ", len(distribuidor_par_unique),
      "\n -> Empresas distintas: ", len(empresas_distintas))

 Cantidad de Comunas evaluadas en df_clientes:  269 
 Cantidad de Comunas en df_par (información CNE):  330 
 -> Comunas distintas:  61

 Cantidad de Distribuidoras evaluadas en df_clientes:  20 
 Cantidad de Distribuidoras en df_par (información CNE):  28 
 -> Empresas distintas:  8


In [11]:
# Cantidad de pares Comuna-Empresa que contiene cada dataframe
pares_clientes = df_clientes_norm[["Comuna", "Distribuidor"]].drop_duplicates()

print(" -> Cantidad de pares Comuna-Empresa únicos en df_cliente:", len(pares_clientes))
print(" -> Cantidad de pares Comuna-Empresa posibles (info CNE):", len(df_par_comuna_empresadx))

 -> Cantidad de pares Comuna-Empresa únicos en df_cliente: 340
 -> Cantidad de pares Comuna-Empresa posibles (info CNE): 484


In [12]:
# Comunas que están en df_par pero no en df_clientes. Por lo tanto, no serán consideradas en el análisis
print(" -> Comunas que, en principio, no serán consideradas en el análisis:\n")
pprint(sorted([c for c in comunas_distintas if pd.notna(c)]), compact=True, width=120)

# Empresas que están en df_par pero no en df_clientes. Por lo tanto, no serán consideradas en el análisis
print("\n -> Distribuidoras que, en principio, no serán considradas en el análisis:\n")
pprint(sorted([e for e in empresas_distintas if pd.notna(e)]), compact=True, width=120)

 -> Comunas que, en principio, no serán consideradas en el análisis:

['aisen', 'alto biobio', 'ancud', 'angol', 'antuco', 'arauco', 'calbuco', 'canete', 'carahue', 'castro', 'chaiten',
 'chile chico', 'cholchol', 'chonchi', 'cisnes', 'cochamo', 'cochrane', 'coihaique', 'collipulli', 'contulmo', 'corral',
 'curaco de velez', 'curanilahue', 'dalcahue', 'futaleufu', 'hualaihue', 'lago verde', 'lanco', 'lebu', 'lonquimay',
 'los alamos', 'los sauces', 'lota', 'lumaco', 'mariquina', 'melipeuco', 'negrete', 'osorno', 'palena', 'panguipulli',
 'puerto octay', 'puqueldon', 'puren', 'putre', 'puyehue', 'queilen', 'quellon', 'quemchi', 'quilaco', 'quinchao',
 'renaico', 'rio ibanez', 'rio negro', 'saavedra', 'san juan de la costa', 'san rosendo', 'santa juana',
 'teodoro schmidt', 'tirua', 'tolten', 'valdivia']

 -> Distribuidoras que, en principio, no serán considradas en el análisis:

['coopersol', 'desa', 'edelaysen', 'frontel', 'luz andes', 'luz osorno', 'mataquito', 'saesa']


### Unificación de los datos

In [ ]:
# Unificación de los datos df_clientes_clean y df_par_comuna_empresadx (6,826,251 registros en total)
df_clientes_par_merged = pd.merge(df_par_comuna_empresadx, df_clientes_norm, 
                                  on=["Comuna", "Distribuidor"], how="right")

df_clientes_par_merged

,Comuna,Distribuidor,Densidad,Mapeo_Densidad,EmpalmeCodigo,ClienteVigencia,ClienteCalidadJuridica,ConsumoTipoUsuario,ConsumoTipoConsumo,ConsumoTipoSuministro,ConsumoTipoTarifa,ConsumoTension,ConsumoPotencia,ConsumoFacturado,ConexionAlimentador,ConexionSubestacionPrimariaDistribucion,id_usuario
0,natales,edelmag,BAJA,3.0,101693682,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,44.355,ALIMENTADOR 2 6,PUERTO NATALES,0
1,natales,edelmag,BAJA,3.0,101693682,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,10.645,ALIMENTADOR 2 6,PUERTO NATALES,1
2,natales,edelmag,BAJA,3.0,101694022,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,1.936,ALIMENTADOR 2 4,PUERTO NATALES,2
3,natales,edelmag,BAJA,3.0,101694024,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,4.167,ALIMENTADOR 2 4,PUERTO NATALES,3
4,natales,edelmag,BAJA,3.0,101836414,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,92.742,ALIMENTADOR 2 2,PUERTO NATALES,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758789,tiltil,electrica til til,MUY BAJA,2.0,56229,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,16.0,709.000,RUNGUE,RUNGUE,7951469
6758790,tiltil,electrica til til,MUY BAJA,2.0,56232,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,3.0,275.000,TILTIL,POLPAICO (ENEL),7951470
6758791,tiltil,electrica til til,MUY BAJA,2.0,56233,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,32.0,0.000,TILTIL,POLPAICO (ENEL),7951471
6758792,tiltil,electrica til til,MUY BAJA,2.0,56244,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,13kV,15.0,17.000,CALEU,CALEU,7951472


## Limpieza de datos de usuarios de la red de distribución

### Cantidad de clientes y energía facturada en distribución (**total**)

Se considerará este valor para obtener la cantidad total de medidores que se requieren cambiar.

#### Filtros para obtener al cliente residencial

In [ ]:
# Tipo de Suministro 
print(df_clientes['ConsumoTipoSuministro'].value_counts())

In [ ]:
# Tipo de Usuario
print(df_clientes['ConsumoTipoUsuario'].value_counts())

In [ ]:
# Tipo de Consumo
print(df_clientes['ConsumoTipoConsumo'].value_counts())

In [ ]:
# Se eliminan los registros que contienen nulos en todas las columnas de interés al mismo tiempo
df_clientes_residenciales = df_clientes.dropna(subset=['ConsumoTipoTarifa', 'ConsumoTipoSuministro', 'ConsumoTipoUsuario', 
                                            'ConsumoTipoConsumo', 'ConsumoFacturado'], how='all')

# Se elimina de la columna Tipo de Tarifa:
# - Usuarios con tarifa no residencial (distintas a BT1a y BT1b) para empresas distintas a "enel distribucion"
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['Distribuidor'] == 'enel distribucion') |
                          (((df_clientes_residenciales['ConsumoTipoTarifa'] == 'BT1a') |
                            (df_clientes_residenciales['ConsumoTipoTarifa'] == 'BT1b')) &
                            (df_clientes_residenciales['Distribuidor'] != 'enel distribucion')) |
                          (df_clientes_residenciales['ConsumoTipoTarifa'].isna())]

# Se eliminan los Clientes Libres de la columna Tipo de Suministro
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoTipoSuministro'] == 'Cliente Regulado') |
                            (df_clientes_residenciales['ConsumoTipoSuministro'].isna())]

# Se eliminan los clientes no-exclusivamente regulados de la columna Tipo de Usuario
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoTipoUsuario'] == 'Cliente Exclusivo Regulado') |
                            (df_clientes_residenciales['ConsumoTipoUsuario'].isna())]

# Se eliminan los clientes no-residenciales de la columna Tipo de Consumo
df_clientes_residenciales = df_clientes_residenciales[((df_clientes_residenciales['ConsumoTipoConsumo'] == 'Residencial') |
                            (df_clientes_residenciales['ConsumoTipoConsumo'] == 'Residencial con Negocio')) |
                            (df_clientes_residenciales['ConsumoTipoConsumo'].isna())]

# Se elimina de la columna Consumo Facturado:
# - Usuarios generadores (fuera del rango -14400 kWh a 0 kWh, para considerar usuarios con generación distribuida [~20 kW])
# - Usuarios con consumo mayor a 7200 kWh/mes (considerando 720 horas en el mes, y 10 kW como máximo en consumo residencial)
df_clientes_residenciales = df_clientes_residenciales[((-14400 < df_clientes_residenciales['ConsumoFacturado']) & 
                            (df_clientes_residenciales['ConsumoFacturado'] < 7200)) |
                            (df_clientes_residenciales['ConsumoFacturado'].isna())]

# Se elimina de la columna Consumo Potencia:
# - Potencia por sobre los 10 kW (máximo en consumo residencial)
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoPotencia'] <= 10) |
                            (df_clientes_residenciales['ConsumoPotencia'].isna())]

In [ ]:
df_clientes_no_residenciales = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_residenciales["id_usuario"])]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_en_dx = df_clientes_no_residenciales[(df_clientes_no_residenciales['ConsumoFacturado'] > 216_000)]

df_clientes_no_residenciales = df_clientes_no_residenciales[~df_clientes_no_residenciales["id_usuario"].isin(df_clientes_libres_en_dx["id_usuario"])]

cantidad_clientes_regulados = len(df_clientes_residenciales) + len(df_clientes_no_residenciales)
cantidad_energia_regulados = df_clientes_residenciales['ConsumoFacturado'].sum() + df_clientes_no_residenciales['ConsumoFacturado'].sum()

In [ ]:
# Se corrigen registros mal clasificados como clientes no residenciales
df_clientes_residenciales_no_evidentes = df_clientes_no_residenciales[df_clientes_no_residenciales['ConsumoTipoConsumo'] == 'Residencial']

# Filtro por Consumo Facturado y Consumo Potencia
df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[((-14400 < df_clientes_residenciales_no_evidentes['ConsumoFacturado']) & 
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'] < 7200)) |
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'].isna())]

df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[(df_clientes_residenciales_no_evidentes['ConsumoPotencia'] <= 10) |
                                                                                (df_clientes_residenciales_no_evidentes['ConsumoPotencia'].isna())]

df_clientes_residenciales = df_clientes_residenciales.merge(df_clientes_residenciales_no_evidentes, how='outer')

df_clientes_no_residenciales = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_residenciales["id_usuario"])]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_en_dx = df_clientes_no_residenciales[(df_clientes_no_residenciales['ConsumoFacturado'] > 216_000)]

df_clientes_no_residenciales = df_clientes_no_residenciales[~df_clientes_no_residenciales["id_usuario"].isin(df_clientes_libres_en_dx["id_usuario"])]

#### Clientes Regulados en distribución (**totales**)

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes.
# -> Cantidad de energía (promedio mensual)
cantidad_clientes_residenciales = len(df_clientes_residenciales)
cantidad_energia_residenciales = df_clientes_residenciales['ConsumoFacturado'].sum()

cantidad_clientes_no_residenciales = len(df_clientes_no_residenciales)
cantidad_energia_no_residenciales = df_clientes_no_residenciales['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
      "\nClientes en distribución:", 
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_regulados), 
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_regulados / 1_000_000_000).round(2)),
      "\nClientes residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000_000).round(2)),
      "\nClientes no residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000_000).round(2)))

#### Clientes Libres en distribución (**totales**)

In [ ]:
cantidad_energia_cliente_libre_dx = df_clientes_libres_en_dx['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
      "\nClientes libres en distribución:",
      "\n -> Cantidad de clientes: {:,}".format(len(df_clientes_libres_en_dx)),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_cliente_libre_dx / 1_000_000_000).round(2)))

### Cantidad de clientes y energía facturada en distribución (**con comuna**)

#### Filtros para obtener al cliente residencial

In [ ]:
# Se eliminan los registros que contienen nulos en todas las columnas de interés al mismo tiempo
df_clientes_par_merged_residencial = df_clientes_par_merged.dropna(subset=['ConsumoTipoTarifa', 'ConsumoTipoSuministro', 'ConsumoTipoUsuario', 
                                            'ConsumoTipoConsumo', 'ConsumoFacturado'], how='all')

# Se elimina de la columna Tipo de Tarifa:
# - Usuarios con tarifa no residencial (distintas a BT1a y BT1b) para empresas distintas a "enel distribucion"
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['Distribuidor'] == 'enel distribucion') |
                                                (((df_clientes_par_merged_residencial['ConsumoTipoTarifa'] == 'BT1a') |
                                                    (df_clientes_par_merged_residencial['ConsumoTipoTarifa'] == 'BT1b')) &
                                                    (df_clientes_par_merged_residencial['Distribuidor'] != 'enel distribucion')) |
                                                (df_clientes_par_merged_residencial['ConsumoTipoTarifa'].isna())]

# Se eliminan los Clientes Libres de la columna Tipo de Suministro
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoTipoSuministro'] == 'Cliente Regulado') |
                            (df_clientes_par_merged_residencial['ConsumoTipoSuministro'].isna())]

# Se eliminan los clientes no-exclusivamente regulados de la columna Tipo de Usuario
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoTipoUsuario'] == 'Cliente Exclusivo Regulado') |
                            (df_clientes_par_merged_residencial['ConsumoTipoUsuario'].isna())]

# Se eliminan los clientes no-residenciales de la columna Tipo de Consumo
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[((df_clientes_par_merged_residencial['ConsumoTipoConsumo'] == 'Residencial') |
                            (df_clientes_par_merged_residencial['ConsumoTipoConsumo'] == 'Residencial con Negocio')) |
                            (df_clientes_par_merged_residencial['ConsumoTipoConsumo'].isna())]

# Se elimina de la columna Consumo Facturado:
# - Usuarios generadores (fuera del rango -14400 kWh a 0 kWh, para considerar usuarios con generación distribuida [~20 kW])
# - Usuarios con consumo mayor a 7200 kWh/mes (considerando 720 horas en el mes, y 10 kW como máximo en consumo residencial)
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[((-14400 < df_clientes_par_merged_residencial['ConsumoFacturado']) & 
                            (df_clientes_par_merged_residencial['ConsumoFacturado'] < 7200)) |
                            (df_clientes_par_merged_residencial['ConsumoFacturado'].isna())]

# Se elimina de la columna Consumo Potencia:
# - Potencia por sobre los 10 kW (máximo en consumo residencial)
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoPotencia'] <= 10) |
                            (df_clientes_par_merged_residencial['ConsumoPotencia'].isna())]

In [ ]:
df_clientes_par_merged_no_residencial = df_clientes_par_merged[~df_clientes_par_merged["id_usuario"].isin(df_clientes_par_merged_residencial["id_usuario"])]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_merged_en_dx = df_clientes_par_merged_no_residencial[(df_clientes_par_merged_no_residencial['ConsumoFacturado'] > 216_000)]

df_clientes_par_merged_no_residencial = df_clientes_par_merged_no_residencial[~df_clientes_par_merged_no_residencial["id_usuario"].isin(df_clientes_libres_merged_en_dx["id_usuario"])]

cantidad_clientes_regulados = len(df_clientes_par_merged_residencial) + len(df_clientes_par_merged_no_residencial)
cantidad_energia_regulados = df_clientes_par_merged_residencial['ConsumoFacturado'].sum() + df_clientes_par_merged_no_residencial['ConsumoFacturado'].sum()

In [ ]:
# Se corrigen registros mal clasificados como clientes no residenciales
df_clientes_residenciales_no_evidentes = df_clientes_par_merged_no_residencial[df_clientes_par_merged_no_residencial['ConsumoTipoConsumo'] == 'Residencial']

# Filtro por Consumo Facturado y Consumo Potencia
df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[((-14400 < df_clientes_residenciales_no_evidentes['ConsumoFacturado']) & 
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'] < 7200)) |
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'].isna())]

df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[(df_clientes_residenciales_no_evidentes['ConsumoPotencia'] <= 10) |
                                                                                (df_clientes_residenciales_no_evidentes['ConsumoPotencia'].isna())]

df_clientes_par_merged_residencial = df_clientes_par_merged_residencial.merge(df_clientes_residenciales_no_evidentes, how='outer')

df_clientes_par_merged_no_residencial = df_clientes_par_merged[~df_clientes_par_merged["id_usuario"].isin(df_clientes_par_merged_residencial["id_usuario"])]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_merged_en_dx = df_clientes_par_merged_no_residencial[(df_clientes_par_merged_no_residencial['ConsumoFacturado'] > 216_000)]

df_clientes_par_merged_no_residencial = df_clientes_par_merged_no_residencial[~df_clientes_par_merged_no_residencial["id_usuario"].isin(df_clientes_libres_merged_en_dx["id_usuario"])]

#### Clientes Regulados (con comuna) en distribución

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes.
# -> Cantidad de energía (promedio mensual)
cantidad_clientes_residenciales = len(df_clientes_par_merged_residencial)
cantidad_energia_residenciales = df_clientes_par_merged_residencial['ConsumoFacturado'].sum()

cantidad_clientes_no_residenciales = len(df_clientes_par_merged_no_residencial)
cantidad_energia_no_residenciales = df_clientes_par_merged_no_residencial['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
      "\nClientes regulados:", 
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_regulados), 
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_regulados / 1_000_000_000).round(2)),
      "\nClientes residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000_000).round(2)),
      "\nClientes no residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000_000).round(2)))

#### Clientes Libres (con comuna) en distribución

In [ ]:
cantidad_energia_cliente_libre_dx = df_clientes_libres_merged_en_dx['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
      "\nClientes libres en distribución:",
      "\n -> Cantidad de clientes: {:,}".format(len(df_clientes_libres_merged_en_dx)),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_cliente_libre_dx / 1_000_000_000).round(2)))

### Caracterización de los clientes residenciales

In [ ]:
# Se crea el df con la cantidad de clientes por par Comuna-Empresa
df_caracterizacion_residencial = df_clientes_par_merged_residencial.groupby(["Comuna", "Distribuidor", "Densidad", "Mapeo_Densidad"], 
                                                                            as_index=False).agg(Cantidad_de_clientes=("Comuna", "size"), 
                                                                                            Cantidad_de_alimentadores=("ConexionAlimentador", "nunique"),
                                                                                            Consumo_facturado_total_MWh=("ConsumoFacturado", lambda x: round(x.sum() / 1000, 2)))
df_caracterizacion_residencial

In [ ]:
# Cantidad considerada de pares Comuna-Empresa unicos posterior a la limpieza
pares_clientes_merged = df_clientes_par_merged[["Comuna", "Distribuidor"]].drop_duplicates()

print(" -> Cantidad de pares Comuna-Empresa únicos en df_cliente:", len(pares_clientes_merged))
print(" -> Cantidad de pares Comuna-Empresa posibles (info CNE):", len(df_par_comuna_empresadx))

# Pares Comuna-Empresa que no están siendo evaluados en el análisis
# - Se hace un merge para comparar
# - Se filtran solo los pares que no aparecen en ambos DataFrames
# - Se eliminan la columna auxiliar de merge
df_pares_no_evaludados = df_par_comuna_empresadx.merge(pares_clientes_merged, on=["Comuna", "Distribuidor"], how="outer", indicator=True)
df_pares_no_evaludados = df_pares_no_evaludados[df_pares_no_evaludados["_merge"] != "both"]                                                
df_pares_no_evaludados = df_pares_no_evaludados.drop(columns=["_merge"])

df_pares_no_evaludados

#### Completitud de los pares Comuna-Empresa faltantes en *df_caracterizacion_residencial*

Fuente de los datos: [Página web energía abierta](http://energiaabierta.cl/categorias-estadistica/electricidad/)

##### Primera estimación de los registros faltantes

In [ ]:
# Información a diciembre de 2024 sobre facturación de clientes regulados (CNE)
df_facturacion_clientes_regulados = pd.read_excel("Datos_Dx_procesados/se_facturacion_clientes_regulados (procesados).xlsx")

# Se elimina la columna Año, ya que no aporta información relevante
df_facturacion_clientes_regulados = df_facturacion_clientes_regulados.drop(columns=['Año'])

# Se filtran los clientes residenciales
df_facturacion_clientes_residenciales = df_facturacion_clientes_regulados[df_facturacion_clientes_regulados["Tipo_clientes"] == "Residencial"].copy()
df_facturacion_clientes_residenciales

In [ ]:
# Se estima la proporción de clientes que representa cada empresa en las comunas que son abastecidas por más de una empresa
comunas_multiples_empresas = (df_caracterizacion_residencial.groupby("Comuna")["Distribuidor"]
                                                            .nunique()
                                                            .reset_index()
                                                            .query("Distribuidor > 1")["Comuna"])

# Se filtra para considerar solo las comunas con múltiples empresas
df_comunas_multiples_empresas = df_caracterizacion_residencial[df_caracterizacion_residencial["Comuna"].isin(comunas_multiples_empresas)].copy()

# Se calcula la proporción de clientes
df_comunas_multiples_empresas["Proporcion_clientes"] = (df_comunas_multiples_empresas["Cantidad_de_clientes"] /
                                                        df_comunas_multiples_empresas.groupby("Comuna")["Cantidad_de_clientes"].transform("sum"))

# Se calcula la proporción de energía
df_comunas_multiples_empresas["Proporcion_energia"] = (df_comunas_multiples_empresas["Consumo_facturado_total_MWh"] /
                                                        df_comunas_multiples_empresas.groupby("Comuna")["Consumo_facturado_total_MWh"].transform("sum"))

proporcion_media = (df_comunas_multiples_empresas.groupby("Densidad")
                                                 .agg(Proporcion_clientes=("Proporcion_clientes", "mean"),
                                                      Proporcion_energia=("Proporcion_energia", "mean"))
                                                      .reset_index()
                                                      .sort_values("Proporcion_clientes", ascending=False))

proporcion_media

In [ ]:
# Se recorre comunas_distintas para agregar información asociada a comunas faltantes
nuevo_registro_df_residencial = pd.DataFrame()

for comuna in comunas_distintas:
    # Filtrar el DataFrame para la comuna actual
    cantidad_de_clientes = df_facturacion_clientes_residenciales.loc[df_facturacion_clientes_residenciales["Comuna"] == comuna, 
                                                                     "Cantidad_de_clientes"].iloc[0]
    cantidad_de_energía = df_facturacion_clientes_residenciales.loc[df_facturacion_clientes_residenciales["Comuna"] == comuna,
                                                                    "Energia_promedio_mensual_2024_kWh"].iloc[0]
    cantidad_de_alimentadores = 0
    par_comuna_emp = df_par_comuna_empresadx[df_par_comuna_empresadx["Comuna"] == comuna]
    df_estimacion = par_comuna_emp.merge(proporcion_media, on="Densidad", how="left")

    df_estimacion["Proporcion_clientes_norm"] = df_estimacion["Proporcion_clientes"] / df_estimacion["Proporcion_clientes"].sum()
    df_estimacion["Cantidad_de_clientes"] = (df_estimacion["Proporcion_clientes_norm"] * cantidad_de_clientes).round().astype(int)

    df_estimacion["Proporcion_energia_norm"] = df_estimacion["Proporcion_energia"] / df_estimacion["Proporcion_energia"].sum()
    df_estimacion["Consumo_facturado_total_MWh"] = (df_estimacion["Proporcion_energia_norm"] * cantidad_de_energía / 1000).round(2)

    # Crear un nuevo registro
    nuevo_registro = {
        "Comuna": comuna,
        "Distribuidor": df_estimacion.iloc[0]["Distribuidor"],
        "Densidad": df_estimacion.iloc[0]["Densidad"],
        "Mapeo_Densidad": df_estimacion.iloc[0]["Mapeo_Densidad"],
        "Cantidad_de_clientes": df_estimacion.iloc[0]["Cantidad_de_clientes"],
        "Cantidad_de_alimentadores": cantidad_de_alimentadores,
        "Consumo_facturado_total_MWh": df_estimacion.iloc[0]["Consumo_facturado_total_MWh"],
    }
    
    # Agregar el nuevo registro al DataFrame
    nuevo_registro_df_residencial = pd.concat([nuevo_registro_df_residencial, pd.DataFrame([nuevo_registro])], ignore_index=True)

In [ ]:
# Se agregan los nuevos registros al DataFrame original del cliente residencial
df_caracterizacion_residencial = pd.concat([df_caracterizacion_residencial, nuevo_registro_df_residencial], ignore_index=True)
df_caracterizacion_residencial

##### Cantidad de clientes y energía resultante residencial de la Primera estimación

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes
# -> Cantidad de energía
cantidad_clientes_residenciales = df_caracterizacion_residencial['Cantidad_de_clientes'].sum()
cantidad_energia_residenciales = df_caracterizacion_residencial['Consumo_facturado_total_MWh'].sum()

print("(A fecha de junio 2025)",
      "\nClientes residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000).round(2)))

##### Segunda estimación de los registros faltantes

In [ ]:
# Cantidad considerada de pares Comuna-Empresa unicos posterior a la limpieza
pares_df_carac_considerados = df_caracterizacion_residencial[["Comuna", "Distribuidor"]].drop_duplicates()

print(" -> Cantidad de pares Comuna-Empresa únicos en df_caracterizacion_residencial:", len(pares_df_carac_considerados))
print(" -> Cantidad de pares Comuna-Empresa posibles (según información de la CNE):", len(df_par_comuna_empresadx))

# Pares Comuna-Empresa que no están siendo evaluados en el análisis (actualizado)
# - Se hace un merge para comparar
# - Se filtran solo los pares que no aparecen en ambos DataFrames
# - Se eliminan la columna auxiliar de merge
df_pares_no_evaluados = df_par_comuna_empresadx.merge(pares_df_carac_considerados, on=["Comuna", "Distribuidor"], how="outer", indicator=True)
df_pares_no_evaluados = df_pares_no_evaluados[df_pares_no_evaluados["_merge"] != "both"]                                                
df_pares_no_evaluados = df_pares_no_evaluados.drop(columns=["_merge"])

df_pares_no_evaluados

In [ ]:
# Total ya asignado por comuna en la caracterización
asignados_por_comuna = (df_caracterizacion_residencial.groupby("Comuna", as_index=False)[["Cantidad_de_clientes", "Consumo_facturado_total_MWh"]]
                                                      .sum()
                                                      .rename(columns={"Cantidad_de_clientes": "clientes_asignados",
                                                                       "Consumo_facturado_total_MWh": "energia_asignada"}))

# Actualizar la cantidad de clientes y la facturación restando lo ya asignado
df_facturacion_clientes_residenciales_actualizado = (df_facturacion_clientes_residenciales.merge(asignados_por_comuna, on="Comuna", how="left"))

df_facturacion_clientes_residenciales_actualizado["Cantidad_de_clientes"] = (df_facturacion_clientes_residenciales_actualizado["Cantidad_de_clientes"]
                                                                                - df_facturacion_clientes_residenciales_actualizado["clientes_asignados"]).clip(lower=0)
df_facturacion_clientes_residenciales_actualizado["Energia_promedio_mensual_2024_kWh"] = (df_facturacion_clientes_residenciales_actualizado["Energia_promedio_mensual_2024_kWh"]
                                                                                - (df_facturacion_clientes_residenciales_actualizado["energia_asignada"] * 1000)).round(2).clip(lower=0)

# Nos quedamos solo con columnas limpias
df_facturacion_clientes_residenciales_actualizado = (df_facturacion_clientes_residenciales_actualizado.drop(columns=["clientes_asignados", "energia_asignada"]).copy())

# Merge de pares no evaluados con el total de clientes por comuna
df_estimacion = df_pares_no_evaluados.merge(
    df_facturacion_clientes_residenciales_actualizado, 
    on="Comuna", 
    how="left"
)

# Incorporar las proporciones medias según densidad
df_estimacion = df_estimacion.merge(proporcion_media[["Densidad", "Proporcion_clientes", "Proporcion_energia"]], on="Densidad", how="left")

# SE elimina el registro que contiene a la comuna 'la union' pues no contiene información
df_estimacion = df_estimacion.dropna(subset=["Cantidad_de_clientes"])
df_estimacion

In [ ]:
# Se recorre comunas_no_evaluadas para estimar la cantidad de clientes en cada par comuna-empresa
nuevo_registro_df_residencial = pd.DataFrame()

for i, row in df_estimacion.iterrows():
    comuna = row["Comuna"]
    distribuidor = row["Distribuidor"]

    df_estimacion_comuna = df_estimacion[(df_estimacion["Comuna"] == comuna) & (df_estimacion["Distribuidor"] == distribuidor)].copy()
    df_estimacion_auxiliar = df_estimacion[df_estimacion["Comuna"] == comuna].copy()
    cantidad_de_clientes = df_estimacion_comuna["Cantidad_de_clientes"].iloc[0]
    cantidad_de_energía = df_estimacion_comuna["Energia_promedio_mensual_2024_kWh"].iloc[0]
    cantidad_de_alimentadores = 0

    df_estimacion_comuna["Proporcion_clientes_norm"] = df_estimacion_comuna["Proporcion_clientes"] / df_estimacion_auxiliar["Proporcion_clientes"].sum()
    df_estimacion_comuna["Proporcion_energia_norm"] = df_estimacion_comuna["Proporcion_energia"] / df_estimacion_auxiliar["Proporcion_energia"].sum()

    df_estimacion_comuna["Cantidad_de_clientes"] = (df_estimacion_comuna["Proporcion_clientes_norm"] * cantidad_de_clientes).round().astype(int)
    df_estimacion_comuna["Consumo_facturado_total_MWh"] = (df_estimacion_comuna["Proporcion_energia_norm"] * cantidad_de_energía / 1000).round(2)

    # Crear un nuevo registro
    nuevo_registro = {
        "Comuna": comuna,
        "Distribuidor": distribuidor,
        "Densidad": df_estimacion_comuna.iloc[0]["Densidad"],
        "Mapeo_Densidad": df_estimacion_comuna.iloc[0]["Mapeo_Densidad"],
        "Cantidad_de_clientes": df_estimacion_comuna.iloc[0]["Cantidad_de_clientes"],
        "Cantidad_de_alimentadores": cantidad_de_alimentadores,
        "Consumo_facturado_total_MWh": df_estimacion_comuna.iloc[0]["Consumo_facturado_total_MWh"],
    }
    nuevo_registro_df_residencial = pd.concat([nuevo_registro_df_residencial, pd.DataFrame([nuevo_registro])], ignore_index=True)

In [ ]:
df_caracterizacion_residencial = pd.concat([df_caracterizacion_residencial, nuevo_registro_df_residencial], ignore_index=True)
df_caracterizacion_residencial

##### Cantidad de clientes y energía resultante residencial de la Primera estimación

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes
# -> Cantidad de energía
cantidad_clientes_residenciales = df_caracterizacion_residencial['Cantidad_de_clientes'].sum()
cantidad_energia_residenciales = df_caracterizacion_residencial['Consumo_facturado_total_MWh'].sum()

print("(A fecha de junio 2025)",
      "\nClientes residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000).round(2)))

##### Se guarda para su uso en la caracterización de la densidad comunal

In [ ]:
df_caracterizacion_residencial.to_excel("Datos_Dx_procesados/caracterizacion_clientes_residenciales (incompleto).xlsx", index=False)

### Caracterización de los clientes No-Residenciales

In [ ]:
# Se crea el df con la cantidad de clientes por par Comuna-Empresa
df_caracterizacion_no_residencial = df_clientes_par_merged_no_residencial.groupby(["Comuna", "Distribuidor", "Densidad", "Mapeo_Densidad"], 
                                                                            as_index=False).agg(Cantidad_de_clientes=("Comuna", "size"), 
                                                                                            Cantidad_de_alimentadores=("ConexionAlimentador", "nunique"),
                                                                                            Consumo_facturado_total_MWh=("ConsumoFacturado", lambda x: round(x.sum() / 1000, 2)))
df_caracterizacion_no_residencial

#### Completitud de los pares Comuna-Empresa faltantes en *df_caracterizacion_no_residencial*

##### Primera estimación de los nuevos registros

In [ ]:
# Se filtran los clientes no residenciales
df_facturacion_clientes_no_residenciales = df_facturacion_clientes_regulados[df_facturacion_clientes_regulados["Tipo_clientes"] == "No Residencial"].copy()

df_facturacion_clientes_no_residenciales

In [ ]:
# Se estima la proporción de clientes que representa cada empresa en las comunas que son abastecidas por más de una empresa
comunas_multiples_empresas = (df_caracterizacion_no_residencial.groupby("Comuna")["Distribuidor"]
                                                            .nunique()
                                                            .reset_index()
                                                            .query("Distribuidor > 1")["Comuna"])

# Se filtra para considerar solo las comunas con múltiples empresas
df_comunas_multiples_empresas = df_caracterizacion_no_residencial[df_caracterizacion_no_residencial["Comuna"].isin(comunas_multiples_empresas)].copy()

# Se calcula la proporción de clientes
df_comunas_multiples_empresas["Proporcion_clientes"] = (df_comunas_multiples_empresas["Cantidad_de_clientes"] /
                                                        df_comunas_multiples_empresas.groupby("Comuna")["Cantidad_de_clientes"].transform("sum"))

# Se calcula la proporción de energía
df_comunas_multiples_empresas["Proporcion_energia"] = (df_comunas_multiples_empresas["Consumo_facturado_total_MWh"] /
                                                        df_comunas_multiples_empresas.groupby("Comuna")["Consumo_facturado_total_MWh"].transform("sum"))

proporcion_media = (df_comunas_multiples_empresas.groupby("Densidad")
                                                 .agg(Proporcion_clientes=("Proporcion_clientes", "mean"),
                                                      Proporcion_energia=("Proporcion_energia", "mean"))
                                                      .reset_index()
                                                      .sort_values("Proporcion_clientes", ascending=False))

proporcion_media

In [ ]:
# Se recorre comunas_distintas para agregar información asociada a comunas faltantes
df_nuevo_registro_no_res = pd.DataFrame()

for comuna in comunas_distintas:
    # Filtrar el DataFrame para la comuna actual
    cantidad_de_clientes = df_facturacion_clientes_no_residenciales.loc[df_facturacion_clientes_no_residenciales["Comuna"] == comuna, 
                                                                     "Cantidad_de_clientes"].iloc[0]
    cantidad_de_energía = df_facturacion_clientes_no_residenciales.loc[df_facturacion_clientes_no_residenciales["Comuna"] == comuna,
                                                                    "Energia_promedio_mensual_2024_kWh"].iloc[0]
    cantidad_de_alimentadores = 0
    par_comuna_emp = df_par_comuna_empresadx[df_par_comuna_empresadx["Comuna"] == comuna]
    df_estimacion = par_comuna_emp.merge(proporcion_media, on="Densidad", how="left")

    df_estimacion["Proporcion_clientes_norm"] = df_estimacion["Proporcion_clientes"] / df_estimacion["Proporcion_clientes"].sum()
    df_estimacion["Cantidad_de_clientes"] = (df_estimacion["Proporcion_clientes_norm"] * cantidad_de_clientes).round().astype(int)

    df_estimacion["Proporcion_energia_norm"] = df_estimacion["Proporcion_energia"] / df_estimacion["Proporcion_energia"].sum()
    df_estimacion["Consumo_facturado_total_MWh"] = (df_estimacion["Proporcion_energia_norm"] * cantidad_de_energía / 1000).round(2)

    # Crear un nuevo registro
    nuevo_registro = {
        "Comuna": comuna,
        "Distribuidor": df_estimacion.iloc[0]["Distribuidor"],
        "Densidad": df_estimacion.iloc[0]["Densidad"],
        "Mapeo_Densidad": df_estimacion.iloc[0]["Mapeo_Densidad"],
        "Cantidad_de_clientes": df_estimacion.iloc[0]["Cantidad_de_clientes"],
        "Cantidad_de_alimentadores": cantidad_de_alimentadores,
        "Consumo_facturado_total_MWh": df_estimacion.iloc[0]["Consumo_facturado_total_MWh"],
    }
    
    # Agregar el nuevo registro al DataFrame
    df_nuevo_registro_no_res = pd.concat([df_nuevo_registro_no_res, pd.DataFrame([nuevo_registro])], ignore_index=True)

In [ ]:
# Se agregan los nuevos registros al DataFrame original del cliente no residencial
df_caracterizacion_no_residencial = pd.concat([df_caracterizacion_no_residencial, df_nuevo_registro_no_res], ignore_index=True)
df_caracterizacion_no_residencial

##### Cantidad de clientes y energía resultante

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes
# -> Cantidad de energía
cantidad_clientes_no_residenciales = df_caracterizacion_no_residencial['Cantidad_de_clientes'].sum()
cantidad_energia_no_residenciales = df_caracterizacion_no_residencial['Consumo_facturado_total_MWh'].sum()

print("(A fecha de junio 2025)",
      "\nClientes no residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000).round(2)))

##### Segunda estimacion de los registros faltantes

In [ ]:
# Total ya asignado por comuna en la caracterización
asignados_por_comuna = (df_caracterizacion_no_residencial.groupby("Comuna", as_index=False)[["Cantidad_de_clientes", "Consumo_facturado_total_MWh"]]
                                                      .sum()
                                                      .rename(columns={"Cantidad_de_clientes": "clientes_asignados",
                                                                       "Consumo_facturado_total_MWh": "energia_asignada"}))

# Actualizar la cantidad de clientes y la facturación restando lo ya asignado
df_facturacion_clientes_no_residenciales_actualizado = (df_facturacion_clientes_no_residenciales.merge(asignados_por_comuna, on="Comuna", how="left"))

df_facturacion_clientes_no_residenciales_actualizado["Cantidad_de_clientes"] = (df_facturacion_clientes_no_residenciales_actualizado["Cantidad_de_clientes"]
                                                                                - df_facturacion_clientes_no_residenciales_actualizado["clientes_asignados"]).clip(lower=0)
df_facturacion_clientes_no_residenciales_actualizado["Energia_promedio_mensual_2024_kWh"] = (df_facturacion_clientes_no_residenciales_actualizado["Energia_promedio_mensual_2024_kWh"]
                                                                                - (df_facturacion_clientes_no_residenciales_actualizado["energia_asignada"] * 1000)).round(2).clip(lower=0)

# Nos quedamos solo con columnas limpias
df_facturacion_clientes_no_residenciales_actualizado = (df_facturacion_clientes_no_residenciales_actualizado.drop(columns=["clientes_asignados", "energia_asignada"]).copy())

# Merge de pares no evaluados con el total de clientes por comuna
df_estimacion = df_pares_no_evaluados.merge(
    df_facturacion_clientes_no_residenciales_actualizado, 
    on="Comuna", 
    how="left"
)

# Incorporar las proporciones medias según densidad
df_estimacion = df_estimacion.merge(proporcion_media[["Densidad", "Proporcion_clientes", "Proporcion_energia"]], on="Densidad", how="left")

# SE elimina el registro que contiene a la comuna 'la union' pues no contiene información
df_estimacion = df_estimacion.dropna(subset=["Cantidad_de_clientes"])
df_estimacion

In [ ]:
# Se recorre comunas_no_evaluadas para estimar la cantidad de clientes en cada par comuna-empresa
nuevo_registro_df_no_residencial = pd.DataFrame()

for i, row in df_estimacion.iterrows():
    comuna = row["Comuna"]
    distribuidor = row["Distribuidor"]

    df_estimacion_comuna = df_estimacion[(df_estimacion["Comuna"] == comuna) & (df_estimacion["Distribuidor"] == distribuidor)].copy()
    df_estimacion_auxiliar = df_estimacion[df_estimacion["Comuna"] == comuna].copy()
    cantidad_de_clientes = df_estimacion_comuna["Cantidad_de_clientes"].iloc[0]
    cantidad_de_energía = df_estimacion_comuna["Energia_promedio_mensual_2024_kWh"].iloc[0]
    cantidad_de_alimentadores = 0

    df_estimacion_comuna["Proporcion_clientes_norm"] = df_estimacion_comuna["Proporcion_clientes"] / df_estimacion_auxiliar["Proporcion_clientes"].sum()
    df_estimacion_comuna["Proporcion_energia_norm"] = df_estimacion_comuna["Proporcion_energia"] / df_estimacion_auxiliar["Proporcion_energia"].sum()

    df_estimacion_comuna["Cantidad_de_clientes"] = (df_estimacion_comuna["Proporcion_clientes_norm"] * cantidad_de_clientes).round().astype(int)
    df_estimacion_comuna["Consumo_facturado_total_MWh"] = (df_estimacion_comuna["Proporcion_energia_norm"] * cantidad_de_energía / 1000).round(2)

    # Crear un nuevo registro
    nuevo_registro = {
        "Comuna": comuna,
        "Distribuidor": distribuidor,
        "Densidad": df_estimacion_comuna.iloc[0]["Densidad"],
        "Mapeo_Densidad": df_estimacion_comuna.iloc[0]["Mapeo_Densidad"],
        "Cantidad_de_clientes": df_estimacion_comuna.iloc[0]["Cantidad_de_clientes"],
        "Cantidad_de_alimentadores": cantidad_de_alimentadores,
        "Consumo_facturado_total_MWh": df_estimacion_comuna.iloc[0]["Consumo_facturado_total_MWh"],
    }
    nuevo_registro_df_no_residencial = pd.concat([nuevo_registro_df_no_residencial, pd.DataFrame([nuevo_registro])], ignore_index=True)

In [ ]:
df_caracterizacion_no_residencial = pd.concat([df_caracterizacion_no_residencial, nuevo_registro_df_no_residencial], ignore_index=True)
df_caracterizacion_no_residencial

##### Cantidad de clientes y energía resultante

In [ ]:
# Información a junio 2025
# -> Cantidad de clientes
# -> Cantidad de energía
cantidad_clientes_no_residenciales = df_caracterizacion_no_residencial['Cantidad_de_clientes'].sum()
cantidad_energia_no_residenciales = df_caracterizacion_no_residencial['Consumo_facturado_total_MWh'].sum()

print("(A fecha de junio 2025)",
      "\nClientes no residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000).round(2)))